In [1]:
import requests
import pandas as pd

class APIDataAggregator:
    def __init__(self, posts_url, comments_url):
        self.posts_url = posts_url
        self.comments_url = comments_url

    def fetch_data(self, url):
        try:
            response = requests.get(url)
            return response.json()
        except requests.exceptions.RequestException as e:
            raise RuntimeError(f"Failed to fetch data from {url}: {e}")

    def count_comments(self, comments):
        counts = {}
        for comment in comments:
            post_id = comment.get("postId")
            counts[post_id] = counts.get(post_id, 0) + 1
        return counts

    def build_dataframe(self, posts, comment_counts):
        df = pd.DataFrame(posts)
        df["comment_count"] = (df["id"].map(comment_counts).fillna(0).astype(int))
        return df

    def save_csv(self, df, filename):
        df.to_csv(filename, index=False)

    def run(self, output_file, preview_rows=5):
        posts = self.fetch_data(self.posts_url)
        comments = self.fetch_data(self.comments_url)

        comment_counts = self.count_comments(comments)
        df = self.build_dataframe(posts, comment_counts)

        print("\n Data Preview:")
        print(df.head(preview_rows))

        self.save_csv(df, output_file)
        print(f"\nCSV saved as '{output_file}'")


if __name__ == "__main__":
    aggregator = APIDataAggregator(
        posts_url="https://jsonplaceholder.typicode.com/posts",
        comments_url="https://jsonplaceholder.typicode.com/comments"
    )

    aggregator.run("posts_with_comment_counts.csv")



 Data Preview:
   userId  id                                              title  \
0       1   1  sunt aut facere repellat provident occaecati e...   
1       1   2                                       qui est esse   
2       1   3  ea molestias quasi exercitationem repellat qui...   
3       1   4                               eum et est occaecati   
4       1   5                                 nesciunt quas odio   

                                                body  comment_count  
0  quia et suscipit\nsuscipit recusandae consequu...              5  
1  est rerum tempore vitae\nsequi sint nihil repr...              5  
2  et iusto sed quo iure\nvoluptatem occaecati om...              5  
3  ullam et saepe reiciendis voluptatem adipisci\...              5  
4  repudiandae veniam quaerat sunt sed\nalias aut...              5  

CSV saved as 'posts_with_comment_counts.csv'
